In [40]:
import psycopg2 as pg2
import heapq#우선순위 큐

In [41]:
db=pg2.connect(host='localhost', dbname='problem',user='postgres',password='kk940107!!',port=5432)
db.autocommit=True
cursor=db.cursor()

In [42]:
def dijkstra(graph,start,end):
    start=str(start)
    value={i:[float('inf'),start] for i in graph}#초기값(모두 무한대)- 정점 방문하지 않은 상태
    value[start]=[0,start]#시작점 값 0으로 변경, 2번째 값은 경로 추적을 위함.
    heap=[]#우선순위 큐
    heapq.heappush(heap,[value[start][0],start])#[가중치,현재노드]
    
    while len(heap)!=0:
        now_value, now_position=heapq.heappop(heap)#가장 우선순위 값 pop
        if value[now_position][0]<now_value:#비교 할 값이 저장되어 있는 값보다 더 크면 확인하지 않아도 됨.
            continue
        
        for node, node_value in graph[now_position].items():#이웃정점 방문(연결되어 있는)
            score=now_value+node_value#현재 값+이웃노드 가중치
            
            if score<value[node][0]:#지금까지 알려진 최소 값보다 작다면
                value[node]=[score,now_position]#값 업데이트
                heapq.heappush(heap,[score,node])#우선순위 큐에 넣음
    
    path=route_tracking(value,start,end)#경로 추적
    print(path)#경로 출력      
             

In [43]:
def route_tracking(value,start,end):#end 부터 역추적
    end=str(end)
    path=[int(end)]#end부터 시작
    while value[end][1] != start:
        path.append(int(value[end][1]))
        end=value[end][1]
    path.append(int(start))
    path.reverse()
    return "경로는 "+str(path)+" 입니다."

In [44]:
#방법1-쿼리에서 그래프 만들기
cursor.execute(#json 형태로 그래프 생성
    "select json_object_agg(sid, value)\
        from (\
            select sid, json_object_agg(fid,weight) as value\
            from route\
            group by sid\
    )s;")

graph=cursor.fetchall()[0][0]#데이터 읽어오기
start=int(input("출발 노드를 입력하세요: "))
end= int(input("도착 노드를 입력하세요: "))
dijkstra(graph,start,end)

경로는 [1, 1363, 1364, 1366, 1369, 1403, 1401, 1218, 1223, 1225, 1226, 1219, 5] 입니다.


In [45]:
#방법2- 파이썬에서 그래프 만들기
cursor.execute("select * from route;")
result=cursor.fetchall()
graph={}#그래프
for i in result:#그래프 만들기
    if str(i[0]) in graph.keys():
        graph[str(i[0])][str(i[1])]=i[2]
    else:
        graph[str(i[0])]={str(i[1]):i[2]}

start=int(input("출발 노드를 입력하세요: "))
end= int(input("도착 노드를 입력하세요: "))
dijkstra(graph,start,end)

경로는 [1, 1363, 1364, 1366, 1369, 1403, 1401, 1218, 1223, 1225, 1226, 1219, 5] 입니다.
